In [1]:
from ipyparallel import Client
rc = Client(profile='pucv-cluster')
len(rc)

95

In [2]:
bview = rc.load_balanced_view()
dview = rc[:]

In [3]:
with dview.sync_imports():
    import re

importing re on engine(s)


In [4]:
import csv
import pandas as pd
import unicodedata
import unidecode

In [5]:
def getLabels(file):
    labels = []
    df = pd.read_excel(file,sep=";")
    for d in df:
        for f in df[d]:
            if not str(f) in ['nan','--','/']:
                labels.append(f)
    return labels

def strip_accents(text):
    aux_text = text.replace("á","a")
    aux_text = aux_text.replace("é","e")
    aux_text = aux_text.replace("í","i")
    aux_text = aux_text.replace("ó","o")
    aux_text = aux_text.replace("ú","u")
    aux_text = aux_text.replace("ñ","n")
    
    return aux_text
    

In [6]:
verbs = getLabels('DECLARATIVE VERBS FOR SOURCES_FINAL NOV10_LISTO.xlsx')

In [7]:
%%time
strip_accents(verbs[50])

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 11.7 µs


'Estimo'

In [8]:
unidecode.unidecode(verbs[700])

'Fa Yan shita'

In [9]:
ar = dview.push({'labels':verbs,'strip_accents':strip_accents},block=False)
ar.wait_interactive()

  95/95 tasks finished after    0 s
done


In [10]:
#verbs

In [11]:
cFile01 = 'COVID-post_enero-julio_FBK-DEF.csv'
df_messages = pd.read_csv(cFile01, sep=';', error_bad_lines=False, low_memory=False)
df_messages['Message'] = df_messages['Message'].fillna('')
messages = df_messages['Message'].values

ar = dview.push({'messages':messages},block=False)
ar.wait_interactive()

  95/95 tasks finished after   37 s
done


In [12]:
def clean(arr):
    for i in range(len(arr)):
        if len(arr[i])>0:
            newWord=''
            for j in range(len(arr[i])):
                w = ',' if j > 0 else ''
                newWord+=w+arr[i][j]
            arr[i]=newWord
        else:
            arr[i]=''

In [13]:
def isOnIt(word, sentence):
    x = re.findall("\\b({}|{})\\b".format(str(word).capitalize(),str(word.lower())),str(sentence))
    if x:
        return True
    else:
        return False
    sentence = sentence.upper().split()
    if not str(sentence) in ['nan','--','/']:
        if str(word).upper() in sentence: 
            return True
        else:
            return False
        
ar = dview.push({'isOnIt':isOnIt},block=False)
ar.wait_interactive()

  95/95 tasks finished after    0 s
done


In [14]:
def getMatch(messages,labels):
    matches = []
    for i in range(len(messages)):
        words = []
        for j in range(len(labels)):
            if isOnIt(labels[j],messages[i]):
                if not str(labels[j]) in words:
                    words.append(labels[j]) 
        matches.append(words)
    clean(matches)
    return matches

@bview.parallel()
def _get_matches(idx):
    words = []
    for j in range(len(labels)):
        if isOnIt(labels[j],messages[idx]):
            if not str(labels[j]) in words:
                words.append(labels[j]) 
    return words

In [15]:
matches = []
labels = verbs

res = _get_matches.map(range(len(messages)))
res.wait_interactive()
ans = res.get()

matches = ans
"""for i in range(len(messages)):
    words = []
    for j in range(len(labels)):
        if isOnIt(labels[j],messages[i]):
            if not str(labels[j]) in words:
                words.append(labels[j]) 
    matches.append(words)
"""

clean(matches)

304002/304002 tasks finished after  881 s
done


In [ ]:
matches

In [17]:
cFile01 = 'COVID-post_enero-julio_FBK-DEF_final.csv'
df_messages = pd.read_csv(cFile01, sep=';', error_bad_lines=False, low_memory=False)
df_messages['Verbs R2'] = matches
df_messages.to_csv(cFile01.replace("_EM","_EM_NV"),sep=";",index=False)

In [18]:
cFile01 = 'COVID-post_enero-julio_IGRAM-DEF_EM.csv'
df_messages = pd.read_csv(cFile01, sep=';', error_bad_lines=False, low_memory=False)
df_messages['Message'] = df_messages['Message'].fillna('')
messages = df_messages['Message'].values

ar = dview.push({'messages':messages},block=False)
ar.wait_interactive()

  95/95 tasks finished after   14 s
done


In [19]:
matches = []
labels = verbs

res = _get_matches.map(range(len(messages)))
res.wait_interactive()
ans = res.get()

matches = ans
"""for i in range(len(messages)):
    words = []
    for j in range(len(labels)):
        if isOnIt(labels[j],messages[i]):
            if not str(labels[j]) in words:
                words.append(labels[j]) 
    matches.append(words)
"""

clean(matches)

33970/33970 tasks finished after  110 s
done


In [20]:
cFile01 = 'COVID-post_enero-julio_IGRAM-DEF_final.csv'
df_messages = pd.read_csv(cFile01, sep=';', error_bad_lines=False, low_memory=False)
df_messages['Verbs R2'] = matches
df_messages.to_csv(cFile01.replace("_EM","_EM_NV"),sep=";",index=False)

In [21]:
cFile01 = 'COVID-post_enero-julio_TWR-DEF_EM.csv'
df_messages = pd.read_csv(cFile01, sep=';', error_bad_lines=False, low_memory=False)
df_messages['Message'] = df_messages['Message'].fillna('')
messages = df_messages['Message'].values

ar = dview.push({'messages':messages},block=False)
ar.wait_interactive()

  95/95 tasks finished after   38 s
done


In [22]:
matches = []
labels = verbs

res = _get_matches.map(range(len(messages)))
res.wait_interactive()
ans = res.get()

matches = ans
"""for i in range(len(messages)):
    words = []
    for j in range(len(labels)):
        if isOnIt(labels[j],messages[i]):
            if not str(labels[j]) in words:
                words.append(labels[j]) 
    matches.append(words)
"""

clean(matches)

314768/314768 tasks finished after  994 s
done


In [23]:
cFile01 = 'COVID-post_enero-julio_TWR-DEF_final.csv'
df_messages = pd.read_csv(cFile01, sep=';', error_bad_lines=False, low_memory=False)
df_messages['Verbs R2'] = matches
df_messages.to_csv(cFile01.replace("_EM","_EM_NV"),sep=";",index=False)

In [24]:
i

NameError: name 'i' is not defined

In [ ]:
df_messages['Message'].values[i]